In [8]:

import math
import datetime
import json
import swisseph as swe


In [ ]:
"""
Enhanced KP chart generator (Python)

This version:
 - accepts birth time with seconds
 - uses Krishnamurti (KP) ayanamsha via Swiss Ephemeris sidereal mode
 - computes planets (including Rahu/Ketu) and house cusps (Placidus)
 - computes nakshatra, charan (pada), and nested sub-lords up to 3 levels
 - returns a JSON-serializable dict with structure similar to the sample you provided

Requirements:
  pip install pyswisseph
  Download Swiss Ephemeris files (or set ephemeris path) if you want highest accuracy.

Usage (example at bottom):
  date = "1981-10-14"
  time = "12:41:22"  # includes seconds
  lat, lon = 11.73, 78.96
  tz = 5.5

Author: Generated by ChatGPT — adapt freely.
"""

# ---------------------- Configuration --------------------------------
EPHE_PATH = None  # set to your ephemeris files folder if needed
PLANETS = [
    (swe.SUN, 'Sun'),
    (swe.MOON, 'Moon'),
    (swe.MERCURY, 'Mercury'),
    (swe.VENUS, 'Venus'),
    (swe.MARS, 'Mars'),
    (swe.JUPITER, 'Jupiter'),
    (swe.SATURN, 'Saturn'),
    (swe.MEAN_NODE, 'Rahu')  # we'll add Ketu as opposite
]

# Vimshottari sequence and proportions
VIMSHOTTARI_ORDER = ['Ketu','Venus','Sun','Moon','Mars','Rahu','Jupiter','Saturn','Mercury']
VIM_YEARS = [7,20,6,10,7,18,16,19,17]
VIM_TOTAL = sum(VIM_YEARS)
#VIM_PROPORTIONS = [y / VIM_TOTAL for y in VIM_YEARS]
VIM_PROPORTIONS = [y/sum(VIM_YEARS) for y in VIM_YEARS]

# Nakshatra names and their lords (standard sequence starting from Ashwini)
NAK_SHAPES = [
    ('Ashwini','Ketu'),('Bharani','Venus'),('Krittika','Sun'),('Rohini','Moon'),('Mrigashira','Mars'),
    ('Ardra','Rahu'),('Punarvasu','Jupiter'),('Pushya','Saturn'),('Ashlesha','Mercury'),('Magha','Ketu'),
    ('Purva Phalguni','Venus'),('Uttara Phalguni','Sun'),('Hasta','Moon'),('Chitra','Mars'),('Swati','Rahu'),
    ('Vishakha','Jupiter'),('Anuradha','Saturn'),('Jyeshtha','Mercury'),('Mula','Ketu'),('Purva Ashadha','Venus'),
    ('Uttara Ashadha','Sun'),('Shravana','Moon'),('Dhanishta','Mars'),('Shatabhisha','Rahu'),('Purva Bhadrapada','Jupiter'),
    ('Uttara Bhadrapada','Saturn'),('Revati','Mercury')
]

# Sign rulers mapping (1..12 where 1=Aries)
SIGN_RULER = {
    1: 'Mars', 2: 'Venus', 3: 'Mercury', 4: 'Moon', 5: 'Sun', 6: 'Mercury',
    7: 'Venus', 8: 'Mars', 9: 'Jupiter', 10: 'Saturn', 11: 'Saturn', 12: 'Jupiter'
}
SIGN_NAMES = ['Aries','Taurus','Gemini','Cancer','Leo','Virgo','Libra','Scorpio','Sagittarius','Capricorn','Aquarius','Pisces']

# -------------------- Helper functions --------------------------------

def parse_date_time(date_str, time_str):
    y,m,d = [int(x) for x in date_str.split('-')]
    hh,mm,ss = [int(x) for x in time_str.split(':')]
    return y,m,d,hh,mm,ss


def to_julian_day(year, month, day, hour=0, minute=0, second=0):
    ut_hours = hour + minute/60.0 + second/3600.0
    return swe.julday(year, month, day, ut_hours)


def normalize_angle(a):
    a = a % 360.0
    if a < 0:
        a += 360.0
    return a


def sign_from_deg(deg):
    deg = normalize_angle(deg)
    sign_idx = int(deg // 30) + 1
    sign_name = SIGN_NAMES[sign_idx - 1]
    return sign_idx, sign_name

def get_nak_charan_and_pos(sid_deg):
    # Normalize the degree
    sid_deg = sid_deg % 360.0

    nak_size = 360.0 / 27.0  # 13°20′ = 13.333333...
    nak_index = int(sid_deg // nak_size) + 1
    if nak_index > 27:
        nak_index = 27

    nak_name, nak_lord = NAK_SHAPES[nak_index - 1]

    nak_start = (nak_index - 1) * nak_size
    pos_in_nak = sid_deg - nak_start

    # Fix tiny floating negative due to rounding
    if pos_in_nak < 0:
        pos_in_nak += nak_size

    # Each nakshatra has 4 padas
    pada_size = nak_size / 4.0
    charan = int(pos_in_nak // pada_size) + 1
    if charan > 4:
        charan = 4

    return nak_index, nak_name, nak_lord, charan, pos_in_nak, nak_size

# def get_nak_charan_and_pos(sid_deg):
#     nak_size = 360.0 / 27.0
#     nak_index = int(sid_deg // nak_size) + 1
#     nak_name, nak_lord = NAK_SHAPES[nak_index - 1]

#     nak_start = (nak_index - 1) * nak_size
#     pos_in_nak = sid_deg - nak_start

#     #pos_in_nak = sid_deg % nak_size
#     # charan (pada) — 4 padas per nakshatra
#     pada_size = nak_size / 4.0
#     charan = int(pos_in_nak // pada_size) + 1
#     return nak_index, nak_name, nak_lord, charan, pos_in_nak, nak_size


# def find_sub_lord_recursive(pos_in_nak_deg, nak_size, levels=3):
#     """Given degrees into a nakshatra (0..nak_size) return nested sub-lords up to levels.
#     We recursively subdivide the current segment using VIM_PROPORTIONS.
#     Returns list like ['Rahu', 'Saturn', 'Mercury'] (length == levels)
#     """
#     lords = []
#     seg_start = 0.0
#     seg_size = nak_size
#     cur_pos = pos_in_nak_deg
#     for level in range(levels):
#         cumulative = 0.0
#         found = False
#         for i, prop in enumerate(VIM_PROPORTIONS):
#             part = prop * seg_size
#             if cur_pos < cumulative + part or i == len(VIM_PROPORTIONS) - 1:
#                 lord = VIMSHOTTARI_ORDER[i]
#                 lords.append(lord)
#                 # refine for next level
#                 # new segment starts at seg_start + cumulative
#                 seg_start = seg_start + cumulative
#                 cur_pos = cur_pos - cumulative
#                 seg_size = part
#                 found = True
#                 break
#             cumulative += part
#         if not found:
#             # fallback
#             lords.append(VIMSHOTTARI_ORDER[-1])
#             seg_size = seg_size * VIM_PROPORTIONS[-1]
#     return lords

def find_sub_lord_recursive(pos_in_nak_deg, nak_size, levels=3):
    """Return KP sublords recursively up to specified levels."""
    VIMSHOTTARI_ORDER = ['Ketu','Venus','Sun','Moon','Mars','Rahu','Jupiter','Saturn','Mercury']
    VIM_PROPORTIONS_YEARS = [7,20,6,10,7,18,16,19,17]
    total_years = sum(VIM_PROPORTIONS_YEARS)
    VIM_PROPORTIONS = [y / total_years for y in VIM_PROPORTIONS_YEARS]

    lords = []
    cur_pos = pos_in_nak_deg / nak_size  # normalize to 0..1
    for level in range(levels):
        cumulative = 0.0
        for lord, prop in zip(VIMSHOTTARI_ORDER, VIM_PROPORTIONS):
            next_cum = cumulative + prop
            if cur_pos <= next_cum or abs(cur_pos - 1.0) < 1e-6:
                lords.append(lord)
                # map to next level
                cur_pos = (cur_pos - cumulative) / prop  # normalize within this segment
                break
            cumulative = next_cum
    return lords

def is_retrograde(jd, pconst, delta_days=2.0):
    lon1 = swe.calc_ut(jd, pconst)[0][0] if isinstance(swe.calc_ut(jd, pconst)[0], (list,tuple)) else swe.calc_ut(jd, pconst)[0]
    lon2 = swe.calc_ut(jd + delta_days, pconst)[0][0] if isinstance(swe.calc_ut(jd + delta_days, pconst)[0], (list,tuple)) else swe.calc_ut(jd + delta_days, pconst)[0]
    # normalize difference
    d = normalize_angle(lon2 - lon1)
    # if motion backwards more than 180 (i.e. negative real change), treat as retro
    # Better: if d > 180 then actual change is d-360 which is negative
    if d > 180:
        d = d - 360
    return d < 0

# -------------------- Core computation --------------------------------

def compute_kp_json(date_str, time_str, lat, lon, tz_offset_hours=5.5, ayan_mode='KP'):
    """Compute KP JSON dict for given local date/time (with seconds) and location.
    ayan_mode: 'KP' or 'LAHIRI' (we set SWEPY sidereal mode accordingly)
    """
    global JD
    y,m,d,hh,mm,ss = parse_date_time(date_str, time_str)
    # convert local to UT
    local_dt = datetime.datetime(y,m,d,hh,mm,ss)
    ut_dt = local_dt - datetime.timedelta(hours=tz_offset_hours)
    JD = to_julian_day(ut_dt.year, ut_dt.month, ut_dt.day, ut_dt.hour, ut_dt.minute, ut_dt.second)

    # set ephemeris path if provided
    if EPHE_PATH:
        swe.set_ephe_path(EPHE_PATH)

    # set sidereal mode to KP if requested
    if ayan_mode.upper().startswith('KP'):
        try:
            swe.set_sid_mode(swe.SIDM_KRISHNAMURTI, 0, 0)
        except Exception:
            # fallback: leave default and rely on get_ayanamsa_ut
            pass
    else:
        try:
            swe.set_sid_mode(swe.SIDM_LAHIRI, 0, 0)
        except Exception:
            pass

    ayanamsha = swe.get_ayanamsa_ut(JD)

    out = {'ayanamsha': ayanamsha, 'houses': [], 'planets': []}

    # Planets
    for pconst, pname in PLANETS:
        calc = swe.calc_ut(JD, pconst)
        # calc sometimes returns nested structure; handle both
        if isinstance(calc[0], (list,tuple)):
            tropical_lon = calc[0][0]
        else:
            tropical_lon = calc[0]
        tropical_lon = normalize_angle(tropical_lon)
        sid_lon = normalize_angle(tropical_lon - ayanamsha)

        sign_id, sign_name = sign_from_deg(sid_lon)
        sign_lord = SIGN_RULER[sign_id]
        nak_idx, nak_name, nak_lord, charan, pos_in_nak, nak_size = get_nak_charan_and_pos(sid_lon)



        sub_lords = find_sub_lord_recursive(pos_in_nak, nak_size, levels=3)
        # determine retrograde
        retro = is_retrograde(JD, pconst)

        # determine house placement: find which house cusp the planet's sidereal longitude falls into
        # compute cusps to use for house determination
        cusps, ascmc = swe.houses_ex(JD, lat, lon) if hasattr(swe, 'houses_ex') else swe.houses(JD, lat, lon)
        # normalize cusps list to length 12 starting indexes 1..12
        if len(cusps) == 13:
            cusp_list = [cusps[i] for i in range(1,13)]
        elif len(cusps) == 12:
            cusp_list = [cusps[i] for i in range(0,12)]
        else:
            raise ValueError('Unexpected cusps length: %s' % len(cusps))
        # build ranges for houses (from cusp_i to cusp_{i+1})
        house_no = None
        for i in range(12):
            start = normalize_angle(cusp_list[i] - ayanamsha)
            end = normalize_angle((cusp_list[(i+1)%12] - ayanamsha))
            pdeg = sid_lon
            if start <= end:
                if pdeg >= start and pdeg < end:
                    house_no = i+1
                    break
            else:
                # wraps around 360
                if pdeg >= start or pdeg < end:
                    house_no = i+1
                    break
        if house_no is None:
            house_no = 12

        # house_lord is ruler of the sign on that cusp
        cusp_sign_id = int(((normalize_angle(cusp_list[0] - ayanamsha) // 30)) + 1) if len(cusp_list) else None

        out['planets'].append({
            'planet_name': pname,
            'planet_id': list(map(lambda x: x[1], PLANETS)).index(pname),
            'full_degree': round(sid_lon, 6),
            'norm_degree': round(sid_lon % 30, 6),
            'is_retro': bool(retro),
            'sign_id': sign_id,
            'sign_name': sign_name,
            'sign_lord': sign_lord,
            'house': house_no,
            'house_lord': SIGN_RULER[((int((normalize_angle(cusp_list[house_no-1]-ayanamsha)//30))+1))],
            'nakshatra_name': nak_name,
            'nakshatra_id': nak_idx,
            'nakshatra_lord': nak_lord,
            'nakshatra_charan': charan,
            'sub_lord': sub_lords[0],
            'sub_sub_lord': sub_lords[1] if len(sub_lords) > 1 else None,
            'sub_sub_sub_lord': sub_lords[2] if len(sub_lords) > 2 else None
        })

   # Find Rahu data
    rahu = next((p for p in out['planets'] if p['planet_name'] == 'Rahu'), None)
    if rahu:
        ketu_sid_lon = normalize_angle(rahu['full_degree'] + 180)

        # Determine sign, nakshatra etc. same as above
        sign_id, sign_name = sign_from_deg(ketu_sid_lon)
        sign_lord = SIGN_RULER[sign_id]
        nak_idx, nak_name, nak_lord, charan, pos_in_nak, nak_size = get_nak_charan_and_pos(ketu_sid_lon)
        sub_lords = find_sub_lord_recursive(pos_in_nak, nak_size, levels=3)

        # Determine house for Ketu using same cusp logic
        house_no = None
        for i in range(12):
            start = normalize_angle(cusp_list[i] - ayanamsha)
            end = normalize_angle((cusp_list[(i + 1) % 12] - ayanamsha))
            pdeg = ketu_sid_lon
            if start <= end:
                if pdeg >= start and pdeg < end:
                    house_no = i + 1
                    break
            else:
                if pdeg >= start or pdeg < end:
                    house_no = i + 1
                    break
        if house_no is None:
            house_no = 12

        out['planets'].append({
            'planet_name': 'Ketu',
            'planet_id': 100,  # custom ID
            'full_degree': round(ketu_sid_lon, 6),
            'norm_degree': round(ketu_sid_lon % 30, 6),
            'is_retro': rahu['is_retro'],
            'sign_id': sign_id,
            'sign_name': sign_name,
            'sign_lord': sign_lord,
            'house': house_no,
            'house_lord': SIGN_RULER[((int((normalize_angle(cusp_list[house_no - 1] - ayanamsha) // 30)) + 1))],
            'nakshatra_name': nak_name,
            'nakshatra_id': nak_idx,
            'nakshatra_lord': nak_lord,
            'nakshatra_charan': charan,
            'sub_lord': sub_lords[0],
            'sub_sub_lord': sub_lords[1] if len(sub_lords) > 1 else None,
            'sub_sub_sub_lord': sub_lords[2] if len(sub_lords) > 2 else None
        })
     # Houses
    cusps, ascmc = swe.houses_ex(JD, lat, lon) if hasattr(swe, 'houses_ex') else swe.houses(JD, lat, lon)
    if len(cusps) == 13:
        cusp_list = [cusps[i] for i in range(1,13)]
    elif len(cusps) == 12:
        cusp_list = [cusps[i] for i in range(0,12)]
    else:
        raise ValueError('Unexpected cusps length: %s' % len(cusps))

    for i in range(12):
        cusp_trop = normalize_angle(cusp_list[i])
        cusp_sid = normalize_angle(cusp_trop - ayanamsha)

        sign_id, sign_name = sign_from_deg(cusp_sid)
        sign_lord = SIGN_RULER[sign_id]
        nak_idx, nak_name, nak_lord, charan, pos_in_nak, nak_size = get_nak_charan_and_pos(cusp_sid)

        print(f"Cusp degree (sid_lon): {cusp_sid:.6f}")
        print(f"Nakshatra index: {nak_idx}, Name: {nak_name}, Lord: {nak_lord}")
        print(f"Nakshatra starts at: {nak_idx * 13.3333333:.6f}")
        print(f"Position within nakshatra: {pos_in_nak:.6f} / {nak_size:.6f} deg")
        print(f"---")
        
        sub_lords = find_sub_lord_recursive(pos_in_nak, nak_size, levels=3)

        out['houses'].append({
            'house_id': i+1,
            'full_degree': round(cusp_sid, 6),
            'norm_degree': round(cusp_sid % 30, 6),
            'sign_id': sign_id,
            'sign_name': sign_name,
            'sign_lord': sign_lord,
            'nakshatra_id': nak_idx,
            'nakshatra_name': nak_name,
            'nakshatra_lord': nak_lord,
            'nakshatra_charan': charan,
            'sub_lord': sub_lords[0],
            'sub_sub_lord': sub_lords[1] if len(sub_lords) > 1 else None,
            'sub_sub_sub_lord': sub_lords[2] if len(sub_lords) > 2 else None
        })

    return out


In [10]:
def short_name(name):
    """Return 2-letter short name for planets/lords."""
    mapping = {
        'Sun': 'Su', 'Moon': 'Mo', 'Mars': 'Ma', 'Mercury': 'Me',
        'Jupiter': 'Ju', 'Venus': 'Ve', 'Saturn': 'Sa',
        'Rahu': 'Ra', 'Ketu': 'Ke'
    }
    return mapping.get(name, name[:2])

In [11]:
from tabulate import tabulate

def print_kp_table(kpjson):
    """Print KP chart in table format (House and Planet rows separated properly)."""
    houses = sorted(kpjson["houses"], key=lambda x: x["house_id"])
    planets = sorted(kpjson["planets"], key=lambda x: x["house"])

    # group planets by house
    planets_by_house = {}
    for p in planets:
        planets_by_house.setdefault(p["house"], []).append(p)

    rows = []
    for h in houses:
        # ---- House row ----
        h_row = {
            "B#": h["house_id"],
            "SL": short_name(h["sign_lord"]),
            "DAL": short_name(h["nakshatra_lord"]),
            "BHL": short_name(h["sub_lord"]),
            "ANL": short_name(h["sub_sub_lord"]),
            "DEG": f"{h['full_degree']:.2f}",
            "PL": "", "PSL": "", "NL": "", "PSU": "", "SSL": ""
        }
        rows.append(h_row)

        # ---- Planet rows ----
        plist = planets_by_house.get(h["house_id"], [])
        for p in plist:
            prow = {
                "B#": "",
                "SL": "",
                "DAL": "",
                "BHL": "",
                "ANL": "",
                "DEG": f"{p['full_degree']:.2f}",
                "PL": short_name(p["planet_name"]),
                "PSL": short_name(p["sign_lord"]),
                "NL": short_name(p["nakshatra_lord"]),
                "PSU": short_name(p["sub_lord"]),
                "SSL": short_name(p["sub_sub_lord"]),
            }
            rows.append(prow)

    print(tabulate(rows, headers="keys", tablefmt="fancy_grid"))

In [12]:
# -------------------- Example / CLI ----------------------------------
if __name__ == '__main__':
    if EPHE_PATH:
        swe.set_ephe_path(EPHE_PATH)

    # Example input
    date = '1981-10-14'
    time = '2:40:26'  # includes seconds
    lat, lon = 11.73, 78.96
    tz = 5.5

    print('Computing  JSON...')
    kpjson = compute_kp_json(date, time, lat, lon, tz, ayan_mode='KP')
    #print(json.dumps(kpjson, indent=2))
    print_kp_table(kpjson)

Computing  JSON...
Cusp degree (sid_lon): 125.747564
Nakshatra index: 10, Name: Magha, Lord: Ketu
Nakshatra starts at: 133.333333
Position within nakshatra: 5.747564 / 13.333333 deg
---
Cusp degree (sid_lon): 155.367202
Nakshatra index: 12, Name: Uttara Phalguni, Lord: Sun
Nakshatra starts at: 160.000000
Position within nakshatra: 8.700535 / 13.333333 deg
---
Cusp degree (sid_lon): 186.686442
Nakshatra index: 15, Name: Swati, Lord: Rahu
Nakshatra starts at: 200.000000
Position within nakshatra: 0.019776 / 13.333333 deg
---
Cusp degree (sid_lon): 217.535416
Nakshatra index: 17, Name: Anuradha, Lord: Saturn
Nakshatra starts at: 226.666666
Position within nakshatra: 4.202082 / 13.333333 deg
---
Cusp degree (sid_lon): 247.068131
Nakshatra index: 19, Name: Mula, Lord: Ketu
Nakshatra starts at: 253.333333
Position within nakshatra: 7.068131 / 13.333333 deg
---
Cusp degree (sid_lon): 276.155231
Nakshatra index: 21, Name: Uttara Ashadha, Lord: Sun
Nakshatra starts at: 279.999999
Position withi

In [ ]:
nak_idx, nak_name, nak_lord, charan, pos_in_nak, nak_size = get_nak_charan_and_pos(sid_lon)

print(f"Planet degree (sid_lon): {sid_lon:.6f}")
print(f"Nakshatra index: {nak_idx}, Name: {nak_name}, Lord: {nak_lord}")
print(f"Nakshatra starts at: {nak_idx * 13.3333333:.6f}")
print(f"Position within nakshatra: {pos_in_nak:.6f} / {nak_size:.6f} deg")
print(f"---")

In [37]:
import swisseph as swe
import datetime
from math import fmod

# copy helpers from your code (normalize, nakshatra & sub-lord functions)
def normalize_angle(a):
    a = a % 360.0
    if a < 0: a += 360.0
    return a

NAK_SIZE = 360.0 / 27.0
VIM_ORDER = ['Ketu','Venus','Sun','Moon','Mars','Rahu','Jupiter','Saturn','Mercury']
VIM_YEARS = [7,20,6,10,7,18,16,19,17]
VIM_PROP = [y/sum(VIM_YEARS) for y in VIM_YEARS]

def get_nak_and_subs(sid_deg):
    nak_idx = int(sid_deg // NAK_SIZE) + 1
    pos_in_nak = sid_deg % NAK_SIZE
    # find sub-lord via cumulative proportions
    cumulative = 0.0
    subs = []
    seg_start = 0.0
    seg_size = NAK_SIZE
    cur_pos = pos_in_nak
    for level in range(3):
        cum = 0.0
        for i, p in enumerate(VIM_PROP):
            part = p * seg_size
            if cur_pos < cum + part or i == len(VIM_PROP)-1:
                subs.append(VIM_ORDER[i])
                # refine for next level
                seg_start = seg_start + cum
                cur_pos = cur_pos - cum
                seg_size = part
                break
            cum += part
    return nak_idx, pos_in_nak, subs

def compare(date, time, lat, lon, tz):
    # parse local time
    y,m,d = map(int, date.split('-'))
    hh,mm,ss = map(int, time.split(':'))
    local = datetime.datetime(y,m,d,hh,mm,ss)
    ut = local - datetime.timedelta(hours=tz)
    jd = swe.julday(ut.year, ut.month, ut.day, ut.hour + ut.minute/60.0 + ut.second/3600.0)

    print("UTC JD:", jd)

    for mode_name, sidmode in [('LAHIRI', swe.SIDM_LAHIRI), ('KP', swe.SIDM_KRISHNAMURTI)]:
        try:
            swe.set_sid_mode(sidmode, 0, 0)
        except Exception as e:
            print("sid_mode set failed:", e)
        ay = swe.get_ayanamsa_ut(jd)
        print("\n---", mode_name, "ayanamsha:", ay)

        # planets to inspect (example: take the body near 125.75 deg — check each)
        bodies = [
            (swe.SUN, 'Sun'), (swe.MOON, 'Moon'), (swe.MERCURY, 'Mercury'),
            (swe.VENUS, 'Venus'), (swe.MARS, 'Mars'), (swe.JUPITER, 'Jupiter'),
            (swe.SATURN, 'Saturn'),
            (swe.TRUE_NODE, 'Rahu (True)'), (swe.MEAN_NODE, 'Rahu (Mean)')
        ]
        for pid, pname in bodies:
            calc = swe.calc_ut(jd, pid)
            if isinstance(calc[0], (list, tuple)):
                trop = calc[0][0]
            else:
                trop = calc[0]
            trop = normalize_angle(trop)
            sid = normalize_angle(trop - ay)
            nak_idx, pos_in_nak, subs = get_nak_and_subs(sid)
            print(f"{pname:12s}: trop={trop:.6f} sid={sid:.6f} nak#{nak_idx} pos_in_nak={pos_in_nak:.4f} subs={subs}")

# Your inputs — OLD and NEW
date = '1981-10-14'
new_time = '02:10:26'
old_time = '02:40:26'
lat, lon, tz = 11.73, 78.96, 5.5

print('=== NEW time ===')
compare(date, new_time, lat, lon, tz)
print('\n=== OLD time ===')
compare(date, old_time, lat, lon, tz)

=== NEW time ===
UTC JD: 2444891.361412037

--- LAHIRI ayanamsha: 23.602636078557225
Sun         : trop=200.417996 sid=176.815360 nak#14 pos_in_nak=3.4820 subs=['Sun', 'Saturn', 'Venus']
Moon        : trop=24.976788 sid=1.374152 nak#1 pos_in_nak=1.3742 subs=['Venus', 'Sun', 'Mercury']
Mercury     : trop=210.224460 sid=186.621824 nak#14 pos_in_nak=13.2885 subs=['Mercury', 'Mercury', 'Saturn']
Venus       : trop=245.459849 sid=221.857213 nak#17 pos_in_nak=8.5239 subs=['Jupiter', 'Rahu', 'Saturn']
Mars        : trop=145.743495 sid=122.140859 nak#10 pos_in_nak=2.1409 subs=['Venus', 'Jupiter', 'Moon']
Jupiter     : trop=200.678708 sid=177.076072 nak#14 pos_in_nak=3.7427 subs=['Moon', 'Venus', 'Venus']
Saturn      : trop=193.775717 sid=170.173081 nak#13 pos_in_nak=10.1731 subs=['Saturn', 'Mars', 'Jupiter']
Rahu (True) : trop=117.859406 sid=94.256770 nak#8 pos_in_nak=0.9234 subs=['Venus', 'Venus', 'Ketu']
Rahu (Mean) : trop=117.375184 sid=93.772548 nak#8 pos_in_nak=0.4392 subs=['Ketu', 'Rahu'